In [2]:
import os 
import numpy as np
import pandas as pd
from utils.utils import *
import nibabel as nib
from scipy.stats import wilcoxon

In [3]:
model_path = os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_inversed.npy')
model = np.load(model_path, allow_pickle=True)
model.shape

(33797, 9)

In [22]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""


columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)
    for j, roi in enumerate(rois.keys()):
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
       
        scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, j] = scores_per_roi_lh
        results[1, i, j] = scores_per_roi_rh

results[:, 0]
        
        



(2, 8, 12, 12)


array([[[ 3.32431260e-02,  2.62957198e-02,  2.17434852e-02,
          1.20587117e-02,  8.76861622e-03,  8.56351291e-03,
         -3.67543457e-03, -1.10208988e-03, -3.66743731e-04,
         -2.47612027e-03, -1.82840276e-03, -2.80433248e-04],
        [ 2.55351658e-02,  2.21965043e-02,  2.06439289e-02,
          6.24784098e-03,  3.52547123e-03, -3.03241147e-04,
         -4.40239402e-03, -1.69117053e-03,  4.25234715e-03,
         -1.17467085e-03,  4.02479518e-04,  6.08951347e-04],
        [ 2.20140192e-02,  2.59257829e-02,  2.71559116e-02,
          1.82983251e-02,  7.54301454e-03,  8.98253225e-03,
         -2.11239159e-03, -7.41370614e-04,  4.50432202e-03,
         -6.70168814e-04, -2.55327957e-03, -7.47172012e-04],
        [ 2.26664169e-02,  2.79696826e-02,  2.95757549e-02,
          1.19209325e-02,  6.61870452e-03, -6.75151223e-04,
         -3.45920152e-03, -8.94703468e-04,  1.13014056e-02,
          2.81889158e-04,  1.06072891e-03,  1.70517097e-03],
        [ 1.89997692e-02,  2.7532850

In [24]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [25]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.006159,0.013197,0.011803,0.006188,-0.004491,-0.008362,-0.007092,-0.006327,0.000516,-0.004508,-0.000158,-0.000376,V2
V2,0.030054,0.026462,0.023849,0.001388,-0.002515,-0.006414,-0.006345,-0.004847,0.010666,-0.000587,0.000542,0.000373,V1
V3,0.019979,0.035076,0.029437,0.017708,-0.002298,-0.005372,-0.004818,-0.004145,0.011774,0.003130,0.002348,-0.000654,V2
hV4,0.023744,0.034524,0.026778,0.002219,-0.002451,-0.006956,-0.006235,-0.004054,0.024719,0.004418,0.005563,0.002178,V2
VO-1,0.016419,0.028426,0.031096,0.020774,-0.000293,-0.003584,-0.002483,-0.002077,0.016607,0.005119,0.003762,-0.000464,V3
VO-2,0.021211,0.030611,0.031972,0.004618,0.000096,-0.005986,-0.005880,-0.003701,0.031237,0.008778,0.009355,0.003846,V3
PHC-1,0.005832,0.018630,0.022910,0.040554,0.006223,-0.001228,-0.001446,-0.001999,0.011449,0.002021,0.012251,0.003487,hV4
PHC-2,0.002161,0.005589,0.008048,0.008090,0.016420,0.036903,0.048406,0.049857,0.006419,0.005833,0.111117,0.062498,TO-1
LO-1,0.003026,0.004592,0.005773,0.001612,0.009269,0.027603,0.036078,0.047247,0.003043,0.001315,0.052409,0.027868,TO-1
LO-2,0.001889,0.004418,0.008331,0.003766,0.021680,0.040176,0.068090,0.077517,0.007792,0.010345,0.095046,0.065087,TO-1


In [14]:
results.shape

(2, 8, 12, 12)

In [15]:
results[0, 0, 0]

array([-0.05333696, -0.0300514 , -0.01556237, -0.00196099, -0.01018307,
       -0.01066287, -0.00275734, -0.00065352,  0.0001455 , -0.0048377 ,
       -0.01372858, -0.00821951])

In [4]:
"""
This is the correct way, I think: take from each source roi and then check the sampling values 
"""

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)

    fits_subj = np.zeros((len(rois.keys()), n_voxels, len(columns)))
    for j, roi in enumerate(rois.keys()):

        



        
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
        
        fits_subj[j] = fits_matrix
       
       # scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
       # scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

      #  results[0, i, j] = scores_per_roi_lh
       # results[1, i, j] = scores_per_roi_rh\
    print(np.mean(fits_subj[0, :cutoff[0],5][belongs[:cutoff[0]] == 'V1']))
    for k, roi in enumerate(rois.keys()):
        scores_per_roi_lh = np.array([np.mean(fits_subj[k, :cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_subj[k, cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, k] = scores_per_roi_lh
        results[1, i, k] = scores_per_roi_rh
    


(2, 8, 12, 12)
0.033243126006174385
0.03127391114635417
-0.005450302514527701
0.0154336386515519
0.01522661283911032
0.00939469784049114
-0.00072955255971458
0.01313440257321492


In [67]:
result_path = os.path.join(proj_dir, 'results', 'results_bestROI_all.npy')
np.save(result_path, results)

In [5]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [8]:
subj01_results_rh = pd.DataFrame(np.mean(results, axis=0)[0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.019701,0.019746,0.016773,0.009123,0.002139,0.000101,-0.005384,-0.003714,0.000075,-0.003492,-0.000993,-0.000328,V2
V2,0.027794,0.024329,0.022246,0.003818,0.000505,-0.003359,-0.005374,-0.003269,0.007459,-0.000881,0.000472,0.000491,V1
V3,0.020997,0.030501,0.028297,0.018003,0.002623,0.001805,-0.003465,-0.002443,0.008139,0.001230,-0.000103,-0.000701,V2
hV4,0.023205,0.031247,0.028177,0.007070,0.002084,-0.003815,-0.004847,-0.002474,0.018010,0.002350,0.003312,0.001942,V2
VO-1,0.017709,0.027979,0.031783,0.020699,0.005007,0.004631,-0.001318,-0.000545,0.011433,0.003087,0.000778,-0.000203,V3
VO-2,0.025169,0.033143,0.037466,0.012304,0.005973,-0.000603,-0.003710,-0.001454,0.024462,0.006217,0.005596,0.003255,V3
PHC-1,0.005409,0.015119,0.020334,0.028646,0.008656,0.004000,-0.000574,-0.000430,0.007630,0.002055,0.005938,0.001982,hV4
PHC-2,0.004082,0.007854,0.010687,0.009303,0.030883,0.037592,0.041449,0.046409,0.008453,0.019697,0.091323,0.062881,TO-1
LO-1,0.005290,0.007753,0.008937,0.005350,0.019135,0.034970,0.026933,0.035576,0.004263,0.009650,0.045314,0.030047,TO-1
LO-2,0.001444,0.003213,0.006093,0.003728,0.022737,0.032781,0.051786,0.059262,0.010037,0.018777,0.077092,0.061655,TO-1


In [11]:
"""
Here, collapse hemis and fill the diagonal with -1, except V1's row 

then we can access each particiants from the first index; if we want to average over all participants, we can just collapse that dimension 
"""


results_both_hemis = np.mean(results, axis=0)  # collapse
for i in range(results_both_hemis.shape[0]):
    np.fill_diagonal(results_both_hemis[i, 1:, 1:], -1)
results_both_hemis.shape
result_bothhemis_path = os.path.join(proj_dir, 'results', 'results_bestROI_hemis_collapsed.npy')
np.save(result_bothhemis_path, results_both_hemis)

In [9]:
subj01_results = pd.DataFrame(results_both_hemis[1], columns=rois.keys(), index=rois.keys())
subj01_results["best_roi"]  = subj01_results.idxmax(axis=1)
subj01_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.054127,0.018787,0.020574,0.014294,0.002971,-0.003215,-0.005333,-0.002084,0.002582,-0.004891,-0.003283,-0.002925,V1
V2,0.057340,-1.000000,0.022865,0.003796,0.004268,-0.003892,-0.002609,0.001120,-0.000669,-0.001306,0.003177,0.000554,V1
V3,0.023574,0.031343,-1.000000,0.035251,0.004707,0.000258,-0.004481,-0.000631,0.002640,-0.002583,-0.004028,-0.002940,hV4
hV4,0.021588,0.019198,0.019675,-1.000000,-0.005416,-0.007551,-0.007219,-0.003863,0.011522,-0.001024,-0.004750,-0.003285,V1
VO-1,0.020529,0.022031,0.026052,0.038434,-1.000000,0.000828,-0.001900,0.002494,0.007523,-0.001213,-0.003349,-0.001724,hV4
VO-2,0.032614,0.022989,0.034706,0.008469,0.003559,-1.000000,-0.004447,-0.001502,0.019165,0.005300,-0.000514,-0.002332,V3
PHC-1,0.016565,0.016803,0.023664,0.062324,0.013889,0.004868,-1.000000,0.002690,0.004272,-0.001348,0.000655,-0.000802,hV4
PHC-2,0.011709,0.005612,0.010892,0.023945,0.060820,0.039860,0.038795,-1.000000,0.016164,0.011687,0.061348,0.041765,TO-1
LO-1,0.010074,0.007488,0.010650,0.016560,0.045119,0.055389,0.048851,0.061536,-1.000000,0.016749,0.062513,0.056477,TO-1
LO-2,0.006085,0.003252,0.005699,0.007293,0.038482,0.041907,0.049517,0.059009,0.024158,-1.000000,0.067339,0.060524,TO-1


In [66]:
def compare_best_to_others(matrix):
    z_values = np.zeros((matrix.shape[0]))
    p_values = np.zeros((matrix.shape[0]))

    for i, row in enumerate(matrix):
        max_value = np.max(row)
        print(max_value)


        results= wilcoxon(row-max_value + 1, method='approx') # +1 because other it contains 0; seems wrong. What I do here seems wrong 
        z_values[i] = results.zstatistic
        p_values[i] = results.pvalue

    return z_values, p_values

# Perform the test on the data matrix
z_df, p_df = compare_best_to_others(results_both_hemis[5])
print(z_df.shape)
current_sub= pd.DataFrame(results_both_hemis[5], columns=rois.keys(), index=rois.keys())
current_sub["best_roi"]  = current_sub.idxmax(axis=1)
current_sub['zvalue'] = z_df
current_sub['pvalue'] = p_df
current_sub

0.012164326418366199
0.014982106171091912
0.008819423182404227
0.008012157096213173
0.005773371611479407
0.007540652767008051
-0.003542671771229436
0.017964201229866444
0.06527694325336897
0.07038082945850824
0.06673428111800456
0.015359222434156106
(12,)


,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi,zvalue,pvalue
V1,0.012164,0.010876,0.002780,-0.004522,0.002897,-0.003298,-0.001875,-0.003297,-0.005580,-0.002158,-0.001020,-0.003440,V1,-3.059412,0.002218
V2,0.014982,-1.000000,0.000904,0.001264,-0.000337,-0.002568,-0.002405,-0.000781,-0.002550,0.000996,-0.001648,-0.006606,V1,-2.980965,0.002873
V3,0.008819,0.007649,-1.000000,-0.006387,0.000642,-0.001343,-0.000459,-0.000158,-0.003998,-0.001358,-0.000459,-0.003033,V1,-2.980965,0.002873
hV4,0.006403,-0.001561,-0.007569,-1.000000,0.000664,-0.002951,-0.002513,-0.001844,-0.012601,0.001086,0.008012,0.002558,TO-1,-2.980965,0.002873
VO-1,0.002445,0.004732,0.000291,-0.009834,-1.000000,0.003492,0.003586,0.005773,-0.015030,-0.005969,0.001737,-0.001707,PHC-2,-2.980965,0.002873
VO-2,-0.010458,-0.014269,-0.020414,-0.014479,-0.005465,-1.000000,0.004324,0.007541,-0.035858,-0.005391,0.005434,-0.004411,PHC-2,-2.980965,0.002873
PHC-1,-0.007188,-0.010052,-0.007509,-0.041328,-0.014616,-0.022485,-1.000000,-0.019579,-0.007767,-0.007668,-0.003543,-0.009276,TO-1,-3.059412,0.002218
PHC-2,-0.000737,-0.000468,-0.001450,-0.003680,0.006309,0.009132,0.012058,-1.000000,-0.007329,0.006991,0.017964,0.007574,TO-1,-2.980965,0.002873
LO-1,-0.000469,0.001421,0.003220,0.004906,0.012804,0.033004,0.034908,0.050163,-1.000000,0.034815,0.065277,0.026728,TO-1,-2.980965,0.002873
LO-2,0.001540,0.003989,0.005146,0.009466,0.015930,0.037481,0.053321,0.070381,0.010544,-1.000000,0.061298,0.027246,PHC-2,-2.980965,0.002873


In [57]:
results_allP = np.mean(results_both_hemis, axis=0)
full_results = pd.DataFrame(results_allP, columns=rois.keys(), index=rois.keys())
full_results["best_roi"]  = full_results.idxmax(axis=1)
full_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.014308,0.012451,0.005573,0.000912,0.001520,-0.001586,-0.004401,-0.003277,-0.006977,-0.006356,-0.002538,-0.002037,V1
V2,0.016105,-1.000000,0.004345,-0.001435,-0.000823,-0.002749,-0.003510,-0.001717,-0.010584,-0.005507,0.000047,-0.000613,V1
V3,0.010051,0.015139,-1.000000,0.004061,0.000796,-0.000428,-0.002007,-0.000973,-0.008774,-0.004703,-0.002068,-0.000944,V2
hV4,0.004486,0.003941,-0.001402,-1.000000,-0.002560,-0.004948,-0.004961,-0.003685,-0.013277,-0.005008,-0.001228,-0.001463,V1
VO-1,0.006706,0.008151,0.003971,0.003001,-1.000000,0.001363,0.000072,0.001936,-0.009735,-0.005695,-0.001103,0.000309,V2
VO-2,-0.000138,-0.000724,-0.006910,-0.008252,-0.003914,-1.000000,-0.002757,-0.000949,-0.022764,-0.006627,-0.000653,-0.001470,V1
PHC-1,0.001402,0.002448,0.001255,0.001692,0.000608,-0.000194,-1.000000,0.004770,-0.007473,-0.004587,0.001462,0.001932,PHC-2
PHC-2,0.001687,0.002122,0.001079,0.004942,0.019582,0.022184,0.026418,-1.000000,-0.001585,0.009905,0.034950,0.030565,TO-1
LO-1,-0.000896,-0.000247,-0.002711,-0.001803,0.010669,0.024115,0.026166,0.035203,-1.000000,0.009693,0.040010,0.037495,TO-1
LO-2,-0.001518,-0.000532,-0.001073,0.000948,0.013004,0.024397,0.032699,0.044865,0.000303,-1.000000,0.043284,0.040157,PHC-2


In [71]:
data = np.array([1.2, 1.5, 1.3, 1.8, 1.6, 1.4, 1.9, 1.7, 1.1, 1.4, 1.5, 2.5])

value_to_compare = np.max(data)

# Perform the Wilcoxon signed-rank test
results= wilcoxon(data - value_to_compare, method='approx') # exact doesnt work for 0s; approx doesnt work if there's less than 15 elements 
results.pvalue



0.003314452996697005

In [31]:
# I think the low quality of P3 and P7 is messing up everything
results_both_hemis = np.delete(results_both_hemis, 2, axis=0)
results_both_hemis = np.delete(results_both_hemis, 5, axis=0)

In [32]:
results_both_hemis.shape

(6, 12, 12)

In [33]:
results_allP = np.mean(results_both_hemis, axis=0)
full_results = pd.DataFrame(results_allP, columns=rois.keys(), index=rois.keys())
full_results["best_roi"]  = full_results.idxmax(axis=1)
full_results

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.020963,0.016162,0.008179,0.002262,-0.000161,-0.002742,-0.005804,-0.004329,-0.006022,-0.007787,-0.003642,-0.003067,V1
V2,0.023804,-1.000000,0.008463,0.000542,-0.000781,-0.003120,-0.005173,-0.003237,-0.006692,-0.006478,-0.001151,-0.002017,V1
V3,0.015343,0.021402,-1.000000,0.008641,0.000890,-0.000440,-0.003037,-0.001735,-0.003814,-0.005293,-0.003174,-0.002408,V2
hV4,0.010757,0.011410,0.005999,-1.000000,-0.001294,-0.004841,-0.006489,-0.005020,-0.004261,-0.004875,-0.001704,-0.002565,V2
VO-1,0.009386,0.011861,0.008111,0.005966,-1.000000,0.001216,-0.000965,0.001465,-0.007011,-0.007055,-0.001944,-0.000337,V2
VO-2,0.008284,0.008306,0.004395,-0.001323,-0.000219,-1.000000,-0.002896,-0.000964,-0.009956,-0.005203,-0.000190,-0.002362,V2
PHC-1,0.003972,0.005419,0.006102,0.008513,0.003620,0.001686,-1.000000,0.005239,-0.002200,-0.004708,0.000925,0.001340,hV4
PHC-2,0.003825,0.003911,0.004417,0.009158,0.028415,0.029530,0.033307,-1.000000,0.004795,0.014041,0.042403,0.037715,TO-1
LO-1,0.003710,0.003745,0.003894,0.005046,0.020014,0.033698,0.032938,0.041722,-1.000000,0.014534,0.044976,0.041833,TO-1
LO-2,0.001290,0.001810,0.002616,0.005574,0.021383,0.032484,0.042445,0.053951,0.008421,-1.000000,0.047990,0.044549,PHC-2


In [18]:
check = os.path.join(models_dir, 'best_fits_best_roi_wself_subj03_train_bestroi.npy')
check = np.load(check, allow_pickle=True)
pd.DataFrame(check)

,0,1,2,3,4,5,6,7,8,9,10
0,0.386539,0.809281,0.028307,0.815959,0.000487,0.00002,0.000822,1.118889,0.445594,V1,TO-1
1,-1.05,1.05,0.229583,0.584835,-0.036644,0.003815,0.0103,1.484924,-0.785398,V1,TO-1
2,1.049999,-1.049999,8.806639,8.29443,-8.209409,0.001154,0.00327,1.484924,2.356194,V1,TO-1
3,1.05,-1.05,4.203879,2.424832,-2.317595,0.001694,0.004906,1.484924,2.356194,V1,TO-1
4,1.05,-0.945283,5.287066,4.477399,-4.346155,-0.000572,0.006769,1.484924,2.30376,V1,TO-1
...,...,...,...,...,...,...,...,...,...,...,...
35909,0.163321,-0.831383,0.040673,0.291534,0.009257,-0.000926,0.000105,1.062626,2.947618,PHC-2,TO-1
35910,1.05,1.05,0.317397,0.359173,-0.016527,-0.006747,0.007229,1.484924,0.785398,PHC-2,TO-1
35911,-0.487235,-0.222136,5.922677,0.000014,0.014715,-0.001461,-0.0,1.15754,-1.998555,PHC-2,TO-1
35912,1.05,0.914544,0.688219,0.695639,-0.233199,0.018025,0.068075,1.484924,0.854239,PHC-2,TO-1


In [19]:
np.unique(check[:, -1])

array(['PHC-1', 'PHC-2', 'TO-1', 'TO-2'], dtype=object)

In [20]:
subj01_results = pd.DataFrame(results_both_hemis[2], columns=rois.keys(), index=rois.keys())
subj01_results["best_roi"]  = subj01_results.idxmax(axis=1)
print(np.unique(subj01_results["best_roi"]))
subj01_results

['PHC-1' 'PHC-2' 'TO-1' 'TO-2']


,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.010140,-0.001593,-0.005533,-0.005946,0.001425,-0.001647,-0.000367,-0.000177,-0.013996,-0.003520,0.001553,0.000755,TO-1
V2,-0.003264,-1.000000,-0.005552,-0.005799,0.002101,0.000769,0.003228,0.004225,-0.016507,-0.003074,0.007214,0.004463,TO-1
V3,-0.004892,-0.001647,-1.000000,-0.008002,-0.004272,-0.002082,0.003430,0.002276,-0.022766,-0.004765,0.000993,0.001933,PHC-1
hV4,-0.005817,-0.005736,-0.012469,-1.000000,-0.002194,-0.001203,0.002739,0.002978,-0.023254,-0.004639,0.002750,0.002408,PHC-2
VO-1,0.003600,0.002449,-0.004353,-0.002483,-1.000000,0.003644,0.006068,0.005687,-0.010363,-0.000638,0.003397,0.002057,PHC-1
VO-2,-0.024273,-0.024696,-0.039273,-0.029272,-0.013444,-1.000000,0.000704,0.001434,-0.056798,-0.014758,-0.000383,0.002702,TO-2
PHC-1,-0.005706,-0.003000,-0.013366,-0.017013,-0.000882,0.003493,-1.000000,0.011472,-0.022375,-0.003196,0.008309,0.005288,PHC-2
PHC-2,0.004235,0.004159,0.002842,0.008143,0.010677,0.019358,0.024576,-1.000000,-0.000392,0.004378,0.027824,0.016218,TO-1
LO-1,-0.003560,-0.001841,-0.006914,-0.002862,0.004964,0.024871,0.036728,0.051473,-1.000000,0.004636,0.051637,0.042449,TO-1
LO-2,-0.004425,-0.001272,-0.004767,-0.003442,0.001623,0.024521,0.030050,0.046006,-0.009978,-1.000000,0.041862,0.033674,PHC-2
